In [5]:
# Используем pipeline как высокоуровневый помощник
from transformers import pipeline

# Создаем сообщения в формате диалога
messages = [
    {"role": "user", "content": "Кто ты?"},
]

# Используем модель Phi-4-mini-instruct
pipe = pipeline("text-generation", model="microsoft/Phi-4-mini-instruct", trust_remote_code=True)

try:
    # Генерируем ответ на основе сообщений
    response = pipe(messages)
    print(response[0]['generated_text'])
except Exception as e:
    print(f"Произошла ошибка при генерации текста: {e}")
    print("Попробуйте использовать другую модель, например 'sberbank-ai/rugpt3small_based_on_gpt2'")

Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.28s/it]
Device set to use cpu


[{'role': 'user', 'content': 'Кто ты?'}, {'role': 'assistant', 'content': 'Я Phi, разработанный Microsoft, здесь, чтобы помочь вам с различными задачами. Как я'}]
